In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.core import GraphIndex
from llama_index.core.prompts import PromptTemplate
from llama_index.graph_stores.neo4j import Neo4jGraphStore


Settings.llm = Ollama(model="qwen3-vl")   

In [ ]:


graph_store = Neo4jGraphStore(
    url="bolt://localhost:7687",
    username="neo4j",
    password="tu_password"
)


In [ ]:
graph_index = GraphIndex.from_graph_store(graph_store)

In [ ]:


system_prompt = PromptTemplate(
"""
Eres un psiquiatra experto.  
Utiliza siempre la información proporcionada por Neo4J.
No inventes datos si no existen.
Estructura todas tus respuestas así:
1. Relevancia clínica
2. Evidencia disponible
3. Información faltante
4. Recomendación siguiente
"""
)


In [ ]:
query_engine = graph_index.as_query_engine(
    text_qa_template=system_prompt
)



In [ ]:
response = query_engine.query(
    "¿Qué diagnósticos previos tiene el paciente 123?"
)

print(response)


In [ ]:
response = query_engine.query(
    "Resumen clínico del paciente 501"
)

print(response)


In [ ]:
from llama_index.core.indices.query.query_transform.base import HyDEQueryTransform

query_engine = graph_index.as_query_engine(
    similarity_top_k=5,
)
